# Create Permits, Violations, and TaxParcels File with MGO Data and AS 400 Data.



<b> TO DO: </b>
<ul> <li><del>get values out of the year column to create multiple DFs using those years.</del></li>
<li>also can create a dictionary with those years in there.</li>
<li>use dask to create a lot of DFs for processing.</li>
<li><del>Add GeoSpatial Data to All Permits for Historical Permits.</del></li>
<li><del>Remove Permits from the DB and upload the new Permits from MGO and AS400 Data.</del></li>
<li>Add to Daily Script.</li>
<li>Add YEAR column to AllPermit Feature Class on ESRIDB.</li>
</ul>

<b>Notes:</b>
<ul> 
<li>Permits table on ESRIDB is the dump created from the SQL job "Update Permit Table", this table is very filtered by AS400. This table is used for some old scripts that are going to be deleted. </li></br>
<li>All_Permits on ESRIDB is the dump from AS400. Should not be updated anymore due to Permits no longer going through AS400.Also, contains all the permits in history. </li></br>
<li>HPermit is the processed version of All_Permits bot h are just tables and do not contain any spatial information. </li></br>
<li>Historical Permit is the permits from AS400 with some spatial information. By checking the spatial reference you can filter the table.</li></br>
<li>AllPermit is the matching Parcel #'s from AS400 permits (using Parcel Fabric and ParcelWithSuffix parcels) </li></br>
<li>Current Permit is the permits from AS400 and MGO that are the most recent and  contain spatial information. Does not include Demo Permits and removes the duplicates based on Parcel # </li></br>
<li>The same naming convention applies to the Demo Permits.</li>

</ul>
 


## Import Modules and Declare Global Variables for geoprocessing

In [ ]:
#Use arcNew environment with this notebook.
#Current Location: C:\ProgramData\Anaconda3\envs\arcNew\python.exe
import arcpy as arc
import os
from shutil import copy2
import pandas as pd
import numpy as np
from arcgis import features
from datetime import datetime as dt
from pathlib import Path

In [ ]:
arc.env.overwriteOutput = True
arc.env.outputZFlag = 'Disabled' #To remove z data from parcel fabric due to it being a polygonZ
arc.env.outputMFlag = 'Disabled'
arc.env.qualifiedFieldNames = False
now = dt.now()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'
# print(netDB)

### Only use with notebook

In [ ]:
pd.options.display.max_columns = 40

## For use when in production.

### Create Folders for Permits Review Data

In [ ]:
#Create Folders for Permits Data
pFolder = [f for f in lPath if f.name == 'Processing'][0]
pProcessing = pFolder / 'Permits' / f'{dStr}'
if pProcessing.exists() == True:
    print(f'{pProcessing} already exist.')
else:
    pProcessing.mkdir(parents=True)
    print(f'Created {pProcessing}.')

pFR = [f for f in lPath if f.name == 'Review'][0]
pReview = pFR / 'Permits' / f'{dStr}'
if pReview.exists() == True:
    print(f'{pReview} already exist')
else:
    pReview.mkdir(parents=True)
    print(f'Created {pReview}')

### Create New File GeoDatabase and corresponding Feature Datasets.

In [ ]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / pFolder / f'Data_{mStr}.gdb'
# dsA = gisPath / pFolder / f'{outGDB}.gdb'
locGDB = outGDB / f'Daily_{dStr}'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{pFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

## Create Parcel Fabric DataFrame and Points FC

### Does not work with Parcel Fabric due to bug with 10.5.1

In [ ]:
dbAdd = 'PROD.ADDRESSINGADMIN.Addressing'
dbSufP = 'PROD.GISADMIN.Suffix_Parcels'
dbMPTab = 'PROD.gisadmin.MGOPermits'
dbAPTab = 'PROD.gisadmin.All_Permits'

In [ ]:
#create paths to the data instead of using env.workspace makes it so that I can use arc Walk
arc.env.workspace = f'{iE.parent}'

fcList = [dbAdd, dbSufP]
tList = [dbMPTab, dbAPTab]
cParcels = []
cTables = []
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in fcList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}'
            if arc.Exists(f'{locGDB / b}'):
                cParcels.append(locGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.FeatureClassToFeatureClass_conversion(f, f'{locGDB}', f'{b}')
                cParcels.append(locGDB.joinpath(b))
                print (f'{b} has been copied')
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in tList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}' #Tables need to have a date string since they won't be under the dataset
            # c = locGDB.parent
            if arc.Exists(f'{outGDB / b}'):
                cTables.append(outGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.TableToTable_conversion(f, f'{outGDB}', f'{b}')
                cTables.append(outGDB.joinpath(b))
                print (f'{b} has been copied')
# print(cParcels)
# print(cTables)

### Copy Parcel Fabric to local folder

In [ ]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
pfParcels = 'PROD.MAPPINGADMIN.ParcelFabric_Parcels'
exp = "TYPE = 7 AND Historical = 0"
a = pfParcels.split('.')[-1]
b = f'{a}_{dStr}'
# print(f'{iEPath[1]}')
if arc.Exists(f'{locGDB / b}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{b} already exists')
else:
    print(f'Copying Parcel Fabric to {locGDB}')
    #print(a)
    # g = f'{f}'
    #print(g)
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', b, where_clause=exp)
    cParcels.append(locGDB.joinpath(b))
    print(f'Finished Copying {b} to {locGDB}')

<b>Create points out of Parcel Fabric to be used with Permits.</b>

In [ ]:
#Create Points out of polygons in ParcelFabric
pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
# print(pfFC)
pfPoints = locGDB / f'pfPoints_{dStr}'
# print(pfPoints)
if arc.Exists(f'{pfPoints}'):
    print(f'{pfPoints.name} already exists.')
else:
    arc.FeatureToPoint_management(str(pfFC), f'{pfPoints}', "INSIDE")
    print(f'{pfPoints.name} has been created.')

<b> Get List of Fields from Parcel Fabric, then create DataFrame for processing </b>

In [ ]:
# pfFields = [f.name for f in arc.ListFields(pfFC)]
# print(pfFields)
pf_df = features.GeoAccessor.from_featureclass(pfPoints, fields=['Name'])
pf_df.Name = pf_df.Name.apply(lambda x: x.strip()) #Remove whitespace from the Name Column
print('Parcel Fabric DataFrame created.')
# pf_df.shape

Create DataFrame from Parcel Fabric Polygons

In [ ]:
# pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
ppf_df = features.GeoAccessor.from_featureclass(pfFC, fields=['Name'])
ppf_df.Name = ppf_df.Name.apply(lambda x: x.strip()) #Remove whitespace from the name column
print('Parcel Fabric DataFrame created.')

<b> Create list of columns in DataFrame to check for missing columns. </b>

In [ ]:
# pfC = pf_df.columns.tolist()
# print(pfC)

## MGO Permits DF Creation and Data Curation.

<b> Create DataFrame from MGO Table </b>

In [ ]:
mTab = [f for f in cTables if f.name.startswith('MGOPermits') == True][0]
# mTable = str(tLoc / mgoT)
# print(mTable)
df_mgo = features.GeoAccessor.from_table(mTab, skip_nulls= False)
print(f'{mTab.name} has been created')
# df_mgo.shape

<b> Create DataFrame for Permits Description to use for merge with MGO Permits. </b>

In [ ]:
# move to local GIS Folder for Processing
appCsv = netDir / 'ParcelWithSuffix' / 'Permits_Desc.csv'
locCsv = pProcessing / 'Permits_Desc.csv'
# print(locCsv)
copy2(appCsv, locCsv)
print(f'Copied {locCsv.name} to {locCsv.parent}')

time.sleep(2)

p_desc = pd.read_csv(locCsv, keep_default_na=False)
print('Created Permits Description DataFrame')
# p_desc

In [ ]:
#columns and values have been fixed on the spreadsheet. Does not to be automated due to it never changing.
#not for us in production
# cFix = p_desc.columns.tolist()
# pList = [f.strip() for f in cFix]
# print(pList)

# cpFix = dict(zip(cFix, pList))
# print(cpFix)

# p_desc = p_desc.rename(columns=cpFix)
# # p_desc


Edit the MGO Dates and format them to match AS400 Data.

In [ ]:
# Upper Case all the column names
df_mgo = df_mgo.rename(columns=lambda x: x.upper())

In [ ]:
def fdate(field):
    from datetime import date
    po = date(2021, 1, 1)
    if field >= po:
        return field
    else:
        return ''


In [ ]:
df_mgo.CODATE = df_mgo.CODATE.apply(fdate)
# df_mgo['CODATE'] = pd.to_datetime(df_mgo.CODATE)
# df_mgo.drop(columns='CODate1', inplace=True)

In [ ]:
# df_mgo.CODATE.fillna('', inplace=True)

In [ ]:
df_mgo.ISSUEDATE = df_mgo.ISSUEDATE.apply(fdate)
# df_mgo['ISSUEDATE'] = pd.to_datetime(df_mgo.CODATE)

In [ ]:
df_mgo.APPDATE = df_mgo.APPDATE.astype(str)
df_mgo.ISSUEDATE = df_mgo.ISSUEDATE.astype(str)
df_mgo.CODATE= df_mgo.CODATE.astype(str)

In [ ]:
# df_mgo['CODATE'] = df_mgo['CODATE'].apply(date)
# df_mgo.CODATE.unique()

In [ ]:
df_mgo.ISSUEDATE.replace('NaT', '', inplace=True)

In [ ]:
def date(field):
    if field != '':
        sp = field.split('-')
        return '{}/{}/{}'.format(sp[1], sp[-1], sp[0])
    else:
        return ''

In [ ]:
df_mgo.APPDATE = df_mgo.APPDATE.apply(date)
df_mgo.CODATE = df_mgo.CODATE.apply(date)
df_mgo.ISSUEDATE = df_mgo.ISSUEDATE.apply(date)

In [ ]:
df_mgo.drop(columns='OBJECTID', inplace=True)
# df_mgo

In [ ]:
# df_mgo['PERMITYEAR'] = df_mgo['PERMITNUMBER'].apply(lambda x: x.split('-')[0])
# df_mgo

In [ ]:
df_mgo = df_mgo.rename(columns={'APPTYPE' : 'MGOTYPE', 'PERMITSTATUS': 'MGOSTATUS'})
# df_mgo.columns

In [ ]:
df_mgo['TYPE'] = df_mgo['PERMITNUMBER'].apply(lambda x: x.split('-')[-1])
# df_mgo

In [ ]:
# To find the MGO permits that are not in 2021
# df_mgo_drop = df_mgo.loc[df_mgo['PERMITYEAR'] != "2021", :]
# df_mgo_drop
#drop the permits not in 2021
df_mgo = df_mgo.loc[df_mgo['PERMITYEAR'] == "2021", :].copy(deep=True)
# df_mgo
# df_mgo_drop.to_csv
# df_mgo = df_mgo.loc[df_mgo['PERMITYEAR'] == '2021']
# df_mgo

In [ ]:
# mGODict = df_mgo.to_dict('records')
# print(mGODict)
mCSV = pReview / 'mgoPerms.csv'
df_mgo.to_csv(mCSV, index=False)

Export for Review later to see the outer merge which will return all the values and which tables they are located in.<br>
Merge to get status and Description from Permit Description DataFrame

In [ ]:
#Export for Review later to see the outer merge which will return all the values and which tables they are located in. 
df_mgo_desc_o = df_mgo.merge(p_desc[['TYPE', 'DESCRIPTION']], how='outer', on='TYPE', indicator= True) #merge to create description column
tEr = pReview / 'Type_Error.csv'
df_mgo_desc_o.loc[df_mgo_desc_o._merge == 'left_only', :].to_csv(tEr, index=False) # export csv if any errors
df_mgo_desc_o = df_mgo_desc_o.drop(columns='_merge')

#merge to get status from Permit Description DataFrame
df_mgo_mis = df_mgo_desc_o.merge(p_desc[['MGO_DESC', 'MGO_INIT']], how='outer',
left_on='MGOSTATUS', right_on='MGO_DESC', indicator=True) #merge to create status column
mgoM = pReview / 'Status_Error.csv'
df_mgo_mis.loc[df_mgo_mis._merge == 'left_only'].to_csv(mgoM, index=False)
mgo_df = df_mgo_mis.loc[df_mgo_mis._merge == 'both']
# df_mgo_mis[df_mgo_mis._merge == 'both']
# merMTable = join(locFolder, f'MGOPermitsMerge_{dStr}.csv')
#not for production, used to find null values
# # df_mgo_mis.loc[df_mgo_mis['_merge'] != 'both', :]
# # df_mgo_mis.loc[df_mgo_mis['_merge'] == 'left_only', :]
# df_mgo_mis.to_csv(merMTable, index=False)
# del df_mgo_desc_o
print("Exported MGO Permits Merge Table.")

In [ ]:
# df_mgo_mis[df_mgo_mis.PARCELID == '7-00-10302-01-4300-00001']
# df_mgo_mis.loc[(df_mgo_mis.PARCELID.duplicated() == True) & (df_mgo_mis._merge == 'both'), :].groupby(by='PARCELID')
# print(dList)

## All_Permits DF Creation and Data Curation

<b> Create DataFrame from All_Permits table on Local GDB after exporting from ESRIDB. If arc.env.workspace is changed need to change aPerms to full path.</b>

In [ ]:
aTab = [f for f in cTables if f.name.startswith('All_Permits') == True][0]
# mTable = str(tLoc / mgoT)
# print(mTable)
ap_df = features.GeoAccessor.from_table(aTab, skip_nulls= False)
print(f'{aTab.name} has been created')
# ap_df.shape

In [ ]:
#Remove whitespace and make it upper
ap_df.rename(columns=lambda x: x.strip().upper(), inplace=True)
ap_df.drop(columns='OBJECTID', inplace=True)

Fix a blank entry from PRIDE.

In [ ]:
ap_df['PARCELID'] = ap_df['PARCELID'].apply(lambda x: '7-00-12900-02-5200-00001' if x.strip() == '-00-12900-02-5200-00001' else x)

Create Date columns for sorting by date later.

In [ ]:
def newyear(field):
    if int(field) > 80 and int(field) < 100:
        return "19" + str(field)
    elif int(field) >= 0 and int(field) <=9:
        return "20" + str(field)
    elif int(field) > 9 and int(field) < 80:
        return "20" + str(field)
ap_df['YEAR'] = ap_df['YEAR'].apply(newyear)

In [ ]:
#Prepare AS400 DataFrame for merge
ap_df['PERMITNUMBER'] = ap_df[['YEAR', 'PERMITNUMBER', 'TYPE']].apply(lambda x: f'{x[0]}-{x[1]}-{x[2]}', axis=1)
ap_df = ap_df[ap_df.YEAR != '2021'] #Remove years == 2021
ap_df.drop(columns='DESCRIPTION', inplace=True) #will be merged back using an edited description csv
ap_df_desc = ap_df.merge(p_desc[['TYPE', 'DESCRIPTION']], on='TYPE', how='outer', indicator= True)
ap_df_b = ap_df_desc[ap_df_desc._merge == 'both'].copy(deep=True)
ap_df_b.drop(columns=['PERMITDESC', '_merge'], inplace= True)
ap_df_b.reset_index(drop=True, inplace=True)

In [ ]:
#Create Feature Class for Historical Data, contains some geospatial information. Merge parcel fabric parcels with all permits dataframe (ap_df_b). Keep only values that are matching and ones in the all permits dataframe. Fill in shape and name with zeros for export later
hPFC = locGDB / f'Historical_Permits_{dStr}'
ap_m = pf_df.merge(ap_df_b, how='outer', left_on='Name', right_on='PARCELID', indicator=True)
ap_m = ap_m[ap_m._merge != 'left_only']
ap_m.drop(columns='_merge', inplace=True)
ap_m.SHAPE.fillna(0, inplace=True)
ap_m.Name.fillna(0, inplace=True)
ap_m.reset_index(drop=True, inplace=True)

#Export to CSV for review and drop shape column to add as table to local GDB
apCsv = pReview / 'Permits_Historical.csv'
ap_m.to_csv(apCsv, index=False)
tabAP = ap_m.drop(columns='SHAPE').copy(deep=True)

#Convert Date Columns to datetime
ap_m.APPDATE = pd.to_datetime(ap_m.APPDATE, format='%m/%d/%Y', errors='coerce')
ap_m.DATE = pd.to_datetime(ap_m.DATE, format='%m/%d/%Y', errors='coerce')
ap_m['PERMITYEAR'] = pd.to_datetime(ap_m.YEAR, format='%Y', errors='coerce')
ap_m.CODATE = pd.to_datetime(ap_m.CODATE, format='%m/%d/%Y', errors='coerce')

#Create table with historical permits as a table for the values that don't contain geospatial information.
tabName = locGDB.parent / f'HPermit_{dStr}'
tabAP.spatial.to_table(str(tabName))

#fill in the zeros in SHAPE column with fake spatialReference
fsr = {'x': None, 'y': None, 'spatialReference':{'wkid':26957}}
ap_m.SHAPE = ap_m.SHAPE.apply(lambda x: fsr if x == 0 else x)

#export to feature class
ap_m.spatial.to_featureclass(f'{hPFC}', sanitize_columns=False)

print(f'Exported Historical Permits to {hPFC.name}')

In [ ]:
#Export FC from local GDB to ESRIDB
dbHPerms = netDB / "PROD-GISADMIN.sde" / "PROD.GISADMIN.InspectionandEnforcement" / "PROD.GISADMIN.HistoricalPermit"
# arc.FeatureClassToFeatureClass_conversion(str(hPFC), f'{dbHPerms.parent}', f'{dbHPerms.name}')
arc.TruncateTable_management(str(dbHPerms))
arc.Append_management(str(hPFC), str(dbHPerms), schema_type='NO_TEST')
print(f'Appended {hPFC.name} to {dbHPerms.name}')

#Export table from local GDB to ESRIDB
dbHtab = netDB / "PROD-GISADMIN.sde" / "PROD.GISADMIN.HistPermit"
# arc.TableToTable_conversion(str(tabName), f'{dbHtab.parent}', f'{dbHtab.name}')
arc.TruncateTable_management(str(dbHtab))
arc.Append_management(str(tabName), str(dbHtab), schema_type='NO_TEST')
print(f'Appended {tabName.name} to {dbHtab.name}')

DO NOT USE UNLESS THE PREVIOUS CELL ISN'T EXPORTING

In [ ]:
# Do not use unless ArcGIS module isn't exporting correctly.

# ap_m.loc[ap_m.SHAPE.isnull(), :]
ap_m.spatial.set_geometry('SHAPE', sr=26957)

lal = gisPath / 'HPermits.csv'
ap_m.to_csv(lal, index=False)

import geopandas as gpd
papa = pd.read_csv(lal)
p2p = gpd.GeoDataFrame(papa, geometry='SHAPE', crs=26957)

pmdf = features.GeoAccessor.from_df(ap_m, sr=26957, geometry_column='SHAPE')

In [ ]:
#Remove Duplicate Parcel IDs
dup_ap = ap_df_b.sort_values(by='YEAR')
dup_ap = ap_df_b.drop_duplicates(subset=['YEAR', 'PARCELID', 'TYPE'], keep='last', ignore_index=True)

In [ ]:
#Drop RJ and HD status from the permits
dup_ap = dup_ap.loc[(dup_ap.STATUS != 'RJ') | (dup_ap.STATUS != 'HD')].copy(deep=True)
# dup_ap.STATUS.value_counts()

In [ ]:
dup_ap = dup_ap.merge(p_desc[['AS_DESC', 'AS_INIT']], how='left', left_on='STATUS', right_on='AS_INIT')

In [ ]:
aPerm = pReview / 'AS_Perms.csv'
dup_ap.to_csv(aPerm, index=False)

## Merge MGO Permits and AS400 Permits into one table and get geospatial information from Parcel Fabric Points.

In [ ]:
#Drop and rename columns from AS400 and MGO DataFrames
mgo_df.drop(columns='_merge', inplace=True)
dup_ap.drop(columns='AS_INIT', inplace=True)
dup_ap.rename(columns={'AS_DESC' : 'STATDESC'}, inplace=True)

#Remove Parcel #s that are in AS400 Data (dup_ap)
pmf = mgo_df[~mgo_df.PARCELID.isin(dup_ap.PARCELID)].copy(deep=True)
pmf.rename(columns={'AS_DESC' : 'STATDESC'}, inplace=True)

In [ ]:
#Merge the pmf with dup_ap to get a list of all permits from MGO and AS400
perm_df = pmf.merge(dup_ap, on='PARCELID', how='outer', indicator=True)

In [ ]:
#concat the columns into one.
perm_df['TYPE'] = perm_df.TYPE_x.str.cat(perm_df.TYPE_y, na_rep='')
perm_df['PERMITNUMBER'] = perm_df.PERMITNUMBER_x.str.cat(perm_df.PERMITNUMBER_y, na_rep='')
perm_df['DESCRIPTION'] = perm_df.DESCRIPTION_x.str.cat(perm_df.DESCRIPTION_y, na_rep='')
perm_df['APPDATE'] = perm_df.APPDATE_x.str.cat(perm_df.APPDATE_y, na_rep='')
perm_df['CODATE'] = perm_df.CODATE_x.str.cat(perm_df.CODATE_y, na_rep='')
perm_df['ISSUED'] = perm_df.ISSUEDATE.str.cat(perm_df.DATE, na_rep='')
perm_df['STATUSDESC'] = perm_df.STATDESC.str.cat(perm_df.MGO_DESC, na_rep='')
perm_df['PERMSTATUS'] = perm_df.STATUS.str.cat(perm_df.MGO_INIT, na_rep='')
perm_df['PERMYEAR'] = perm_df.YEAR.str.cat(perm_df.PERMITYEAR, na_rep='')

In [ ]:
#Columns to keep for the Feature Class on ESRIDB and the REST Server
pmCols = ['PARCELID', 'TYPE', 'PERMSTATUS', 'ISSUED', 'PERMYEAR', 'PERMITNUMBER', 'DESCRIPTION', 'SHAPE', 'CODATE', 'APPDATE']
pmFields = [f.name for f in arc.ListFields('PROD.GISADMIN.Permits')][1:]
pmFields.remove('PermitNumb')
kpmCols = dict(zip(pmCols, pmFields))
kpmCols['STATUSDESC'] = 'StatusDesc'
pmCols.append('STATUSDESC')

In [ ]:
pm_df = perm_df.drop(columns=[f for f in perm_df.columns if f not in pmCols])
#Rename Columns
pm_df.rename(columns=kpmCols, inplace=True)

In [ ]:
#convert the date columns to datetime
pm_df.AppDate = pd.to_datetime(pm_df.AppDate, format='%m/%d/%Y', errors='coerce')
pm_df.IssueDate = pd.to_datetime(pm_df.IssueDate, format='%m/%d/%Y', errors='coerce')
pm_df['Year'] = pd.to_datetime(pm_df.PermitYear, format='%Y', errors='coerce')
pm_df.CODate = pd.to_datetime(pm_df.CODate, format='%m/%d/%Y', errors='coerce')

In [ ]:
print(pm_df.columns)
print(pf_df.columns)

In [ ]:
#Merge Parcel Fabric Points with Permit Data
pfm_df = pf_df.merge(pm_df, on='Name', how='outer', indicator=True)
# pfm_df = pm_df.merge(pf_df, on='Name', how='outer', indicator=True)

In [ ]:
#s_pm is all the permits with suffix and t_pm is parcels that aren't in Parcel Fabric
sOut = pReview / 'Permits_Suffix.csv'
tOut = pReview / 'Permits_Parcels_NM.csv' #NM = No Match
s_pm = pfm_df[(pfm_df._merge != 'left_only') & (~pfm_df.Name.str.contains('-00001', na=False))]
t_pm = pfm_df[(pfm_df._merge != 'left_only') & (pfm_df.Name.str.contains('-00001', na=False))]
s_pm.drop(columns='_merge', inplace=True)
s_pm.to_csv(sOut, index=False)
t_pm.to_csv(tOut, index=False)

print(f'Exported {sOut.name} and {tOut.name}')

In [ ]:
#Join suffix permits with parcel with suffix. Merge back into Main Permits afterwards. Need to run this after running the parcel with suffix script.
sParcels = f'{locGDB / f"ParcelWithSuffix_{dStr}"}'
sPar = features.GeoAccessor.from_featureclass(sParcels, fields=['Name'])

#Merge suffix parcels and suffix permits to create a new DF
sufMerge = sPar.merge(s_pm, on='Name', how='outer', indicator=True)
sufPerm = sufMerge[sufMerge._merge == 'both'].copy(deep=True)

Append Suffix Permits and Parcel Permits

In [ ]:
#Prepare the DataFrames for concat
sufPerm.drop(columns='SHAPE_y', inplace=True)
sufPerm.rename(columns={'SHAPE_x':'SHAPE'}, inplace=True)
#Concat suffix and parcels
perm_all = pd.concat([sufPerm, t_pm], axis=0)
print(f'Concatenated DataFrames')

In [ ]:
#Make the PermitYear a year column and Year a string column
perm_all['PermitYear'] = pd.to_datetime(perm_all.Year, format='%Y', errors='coerce')
perm_all.Year = perm_all.Year.astype(str).apply(lambda x: x.split('-')[0])

In [ ]:
#Export to All Permit before dropping the duplicates
permEx = perm_all[perm_all._merge != 'right_only'].copy(deep=True)
permEx.drop(columns='_merge', inplace=True)
permEx.SHAPE.fillna(0, inplace=True)
permEx.reset_index(drop=True, inplace=True)

#Convert Date Columns to datetime
permEx.AppDate = pd.to_datetime(permEx.AppDate, format='%m/%d/%Y', errors='coerce')
# perm_df['PermitYear'] = pd.to_datetime(perm_df.Year, format='%Y', errors='coerce')
permEx.CODate = pd.to_datetime(permEx.CODate, format='%m/%d/%Y', errors='coerce')
# perm_df.Year = perm_df.Year.astype(str)

#Export to CSV for review and drop SHAPE column to add to table
permCSV = pReview / 'AllPermits.csv'
permEx.to_csv(permCSV, index=False)
tabPerm = permEx.drop(columns='SHAPE').copy(deep=True)

#Create table with All Permits as a table for the values that don't contain geospatial information
tabPermN = outGDB / f'AllPTable_{dStr}'
# tabPerm.spatial.to_table(str(tabPermN))
print(f'Exported {tabPermN.name} to {outGDB.name}')

#Fill in the zeros in the SHAPE column with fake spatialReference
permEx.SHAPE = permEx.SHAPE.apply(lambda x: fsr if x == 0 else x)

#Export the dataframe to local GDB
outFC = locGDB / f'AllPermits_{dStr}'
perm_df.spatial.to_featureclass(f'{outFC}', sanitize_columns=False)
print(f'Exported {outFC.name} to {locGDB.name}')

#Export from local GDB to ESRIDB
dbPerms = netDB / "PROD-GISADMIN.sde" / "PROD.GISADMIN.InspectionandEnforcement" / "PROD.GISADMIN.AllPermit"
arc.TruncateTable_management(str(dbPerms))
arc.Append_management(str(outFC), str(dbPerms), schema_type='NO_TEST')
print(f'Appending {outFC.name} to {dbPerms.name}')

In [ ]:
# Export Permits to GDB and append to Permits on the DB.
perm_db = locGDB / f'Permits_DB_{dStr}'
dup_p = perm_all[perm_all._merge == 'both'].copy(deep=True)
demoDrop = dup_p.loc[(dup_p.StructureType == "DEMO") | (dup_p.StructureType == "DEMS") | (dup_p.StructureType == "MHDM")]
dup_p.drop(demoDrop.index, axis=0, inplace=True)
dup_p.reset_index(drop=True, inplace=True)
dup_p.sort_values(by='AppDate', inplace=True)
dup_p.drop_duplicates(subset=['Name'], keep='last', ignore_index=True, inplace=True)
dup_p.drop(columns='_merge', inplace=True)

# #export to Local GBD
dup_p.spatial.to_featureclass(perm_db, sanitize_columns=False)
print(f'Exported {perm_db.name} to {locGDB.name}')

#export to CurrentPermits on ESRIDB
dbCPerms = netDB / "PROD-GISADMIN.sde" / "PROD.GISADMIN.InspectionandEnforcement" / "PROD.GISADMIN.CurrentPermit"
arc.TruncateTable_management(str(dbCPerms))
arc.Append_management(str(perm_db), str(dbCPerms), schema_type='NO_TEST')
print(f'Appended {perm_db.name} to {dbCPerms.name}')

### Create DEMO Permits Feature Class

Create AS400 Demo Permits DataFrame

In [ ]:
#Change Date columns in pm_df
pm_df['PermitYear'] = pd.to_datetime(pm_df.Year, format='%Y', errors='coerce')
pm_df.Year = pm_df.Year.astype(str).apply(lambda x: x.split('-')[0])

In [ ]:
#Export Demo Permits for Review
dCsv = pReview / 'DemoPermits.csv'
df_demo = pm_df.query('StructureType == "DEMO" | StructureType == "DEMS" | StructureType == "MHDM"')
# df_demo.reset_index(drop=True, inplace=True)
df_demo.to_csv(dCsv, index=False)

In [ ]:
#Merge Parcel Fabric Points and df_demo Data
demo_fc = pf_df.merge(df_demo, on='Name', how='outer', indicator=True)

In [ ]:
# export Parcel Names that don't match
dLo = pReview / 'Missing_Demo_P.csv'
demo_fc[demo_fc._merge == 'right_only'].to_csv(dLo, index=False)

In [ ]:
#Create dummy polygon spatial Reference to replace the null values in SHAPE. This will allow you to export to a feature class
locDemo = demo_fc[demo_fc._merge != 'left_only'].copy(deep=True)
locDemo.drop(columns='_merge', inplace=True)
locDemo.SHAPE.fillna(0, inplace=True)
locDemo.reset_index(drop=True, inplace=True)

#Export to CSV for review and drop shape column to add as a table to local gdb
dmCSV = pReview / 'All_Demo_Permits.csv'
locDemo.to_csv(dmCSV, index=False)
tabDemo = locDemo.drop(columns='SHAPE').copy(deep=True)

#Export table to local gdb for values that don't contain geospatial information
tabDN = outGDB / f'AllDPermits_{dStr}'
locDemo.spatial.to_table(str(tabDN))
print(f'Exported {tabDN.name} to {outGDB.name}')

In [ ]:
#Export the Demo Permits DataFrame as a feature class to the local GDB. Not all values contain geospatial information.
# pfsr = {'rings': [[[0,0]]], 'spatialReference': {'wkid': 26957, 'latestWkid': 26957}} #Use if they are polygons
fsr = {'x': None, 'y': None, 'spatialReference':{'wkid':26957}}
#Fill in the zeros in the SHAPE column with fake spatialReference
locDemo.SHAPE = locDemo.SHAPE.apply(lambda x: fsr if x == 0 else x)

#Export the dataframe to local GDB
outDemo = locGDB / f'AllDemoPermits_{dStr}'
locDemo.spatial.to_featureclass(f'{outDemo}', sanitize_columns=False)
print(f'Exported {outDemo.name} to {locGDB.name}')

#Export from local GDB to ESRIDB
dbDPerms = netDB / "PROD-GISADMIN.sde" / "PROD.GISADMIN.InspectionandEnforcement" / "PROD.GISADMIN.AllDemoPermit"
arc.TruncateTable_management(str(dbDPerms))
arc.Append_management(str(outDemo), str(dbDPerms), schema_type='NO_TEST')
print(f'Appending {outDemo.name} to {dbDPerms.name}')

In [ ]:
#Suffix Parcels to be merged with suffix layer
sDem = pReview / 'Suffix_Parcels.csv'
demo_fc[(demo_fc._merge != 'left_only') & (~demo_fc.Name.str.contains('-00001', na=False))].to_csv(sDem, index=False)

#Merge Suffix with demo_fc data
sdem_df = demo_fc[(demo_fc._merge != 'left_only') & (~demo_fc.Name.str.contains('-00001', na=False))].copy(deep=True)
sdem_df.drop(columns=['SHAPE','_merge'], inplace=True)
sdem_df.reset_index(drop=True, inplace=True)
sMD = sPar.merge(sdem_df, how='outer', left_on='Name', right_on='Name', indicator=True)

#Export the missing parcels to review
sufOut = pReview / 'Missing_Suffix.csv'
sMD[sMD._merge == 'right_only'].to_csv(sufOut, index=False)

#Concat the matching into demo_fc
sMD = sMD[sMD._merge == 'both'].copy(deep=True)
dDem = demo_fc[(demo_fc.Name.str.contains('-00001', na=False)) & (demo_fc._merge != 'left_only')].copy(deep=True)
dDem.drop(columns='_merge', inplace=True)
sMD.drop(columns='_merge', inplace=True)
fDemo = [dDem, sMD]
cDem = pd.concat(fDemo)
cDem.reset_index(drop=True, inplace=True)

In [ ]:
#Export the Demo Permits File to Local GDB
demoPerm = locGDB / f'DemoPermits_{dStr}'
cDem.sort_values(by='AppDate', inplace=True)
cDem.drop_duplicates(subset=['Name'], keep='last', ignore_index=True, inplace=True)
cDem.SHAPE.fillna(0,inplace=True)
cDem.IssueDate.fillna('', inplace=True)
cDem.reset_index(drop=True, inplace=True)

#Fill in the zeros in the SHAPE column with fake spatialReference
cDem.SHAPE = cDem.SHAPE.apply(lambda x: fsr if x == 0 else x)

#Remove columns that won't have any values.
cDem.drop(columns='CODate', inplace=True)

#Export to Local GBD
cDem.spatial.to_featureclass(str(demoPerm), sanitize_columns=False)
print(f'Exported {demoPerm.name} to {locGDB.name}')

#export to CurrentPermits on ESRIDB
dbDPerms = netDB / "PROD-GISADMIN.sde" / "PROD.GISADMIN.InspectionandEnforcement" / "PROD.GISADMIN.CurrentDemoPermit"
arc.TruncateTable_management(str(dbDPerms))
arc.Append_management(str(demoPerm), str(dbDPerms), schema_type='NO_TEST')
print(f'Appended {demoPerm.name} to {dbDPerms.name}')